# Recommender system test

### Propósito de la prueba:
Probar cambios relacionados con la introducción de un sistema de recomendaciones mejorado

### Resultado esperado: 
Dentro de los 14 días posteriores a la inscripción, los usuarios mostrarán una mejor conversión en vistas de la página del producto (el evento **product_page**), instancias de agregar artículos al carrito de compras (**product_card**) y compras (**purchase**). En cada etapa del embudo **product_page → product_card → purchase**, habrá al menos un 10% de aumento.

## Instrucciones para completar la tarea

- Describe los objetivos del estudio.
- Explora los datos:
    - ¿Es necesario convertir los tipos?
    - ¿Hay valores ausentes o duplicados? Si es así, ¿cómo los caracterizarías?
- Lleva a cabo el análisis exploratorio de datos:
    - Estudia la conversión en las diferentes etapas del embudo.
    - ¿El número de eventos por usuario está distribuido equitativamente entre las muestras?
    - ¿Hay usuarios que están presentes en ambas muestras?
    - ¿Cómo se distribuye el número de eventos entre los días?
    - ¿Hay alguna peculiaridad en los datos que hay que tener en cuenta antes de iniciar la prueba A/B?
- Evaluar los resultados de la prueba A/B:
    - ¿Qué puedes decir sobre los resultados de la prueba A/B?
    - Utiliza una prueba z para comprobar la diferencia estadística entre las proporciones
- Describe tus conclusiones con respecto a la etapa EDA y los resultados de la prueba A/B

## Describir los objetivos del estudio

Comprobaremos si los usuarios mostraron una mejor conversión en vistas de la página del pruducto (product_page), instancias de agregar artículos al carrito de compras (product_card) y compras (purchase). En cada etapa del embudo product_page → product_card → purchase, habrá al menos un 10% de aumento.

In [2]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import MultiLabelBinarizer
import re

## Exploración de datos

In [3]:
mkt_events = pd.read_csv('datasets/ab_project_marketing_events_us.csv')
users = pd.read_csv('datasets/final_ab_new_users_upd_us.csv')
events = pd.read_csv('datasets/final_ab_events_upd_us.csv')
ab_data = pd.read_csv('datasets/final_ab_participants_upd_us.csv')

datas = [mkt_events, users, events, ab_data]

for data in datas: 
    print('---')
    print(data.sample())
    print()

---
                               name        regions    start_dt   finish_dt
7  Labor day (May 1st) Ads Campaign  EU, CIS, APAC  2020-05-01  2020-05-03

---
                user_id  first_date     region   device
48510  4A7D4DBED54234E8  2020-12-19  N.America  Android

---
                user_id             event_dt    event_name  details
96137  09BB467EF9628A6B  2020-12-20 09:56:44  product_cart      NaN

---
              user_id group                  ab_test
501  5426EFD4BBAEA349     A  recommender_system_test



## MKT events

In [4]:
mkt_events.head()

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


In [5]:
mkt_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


In [6]:
mkt_events.describe()

,name,regions,start_dt,finish_dt
count,14,14,14,14
unique,14,6,14,14
top,Christmas&New Year Promo,APAC,2020-12-25,2021-01-03
freq,1,4,1,1


In [7]:
# Convertimos columnas de fecha a datetime

date_cols = ['start_dt', 'finish_dt']
for date in date_cols: 
    mkt_events[date] = pd.to_datetime(mkt_events[date], errors = 'coerce')

mkt_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 580.0+ bytes


In [8]:
# Buscamos duplicados

mkt_events.duplicated().sum()

np.int64(0)

In [9]:
# Convertimos categorías a columnas binarias

def normalize_region_name(name: str) -> str:
    """
    Normaliza el nombre de región:
    - lower()
    - quita corchetes
    - reemplaza espacios, puntos y comas por "_"
    - elimina dobles guiones bajos
    """
    name = name.lower()
    name = re.sub(r"[\[\]]", "", name)         # quita corchetes
    name = re.sub(r"[ ,.\-]+", "_", name)      # reemplaza separadores por "_"
    name = re.sub(r"_+", "_", name)            # colapsa guiones bajos
    return name.strip("_")                     # elimina guiones bajos extremos

def expand_regions(df, col="regions", sep=","):
    # Dividir en listas, manejando NaN
    regions = df[col].fillna("").apply(
        lambda x: [normalize_region_name(r.strip()) for r in str(x).split(sep) if r.strip()]
    )
    
    # Crear binarizador
    mlb = MultiLabelBinarizer()
    dummies = pd.DataFrame(
        mlb.fit_transform(regions),
        index=df.index,
        columns=mlb.classes_
    )
    
    # Unir al df original
    return df.join(dummies)

mkt_events = expand_regions(mkt_events)

In [10]:
mkt_events

,name,regions,start_dt,finish_dt,apac,cis,eu,n_america
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03,0,0,1,1
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16,1,1,1,1
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19,0,0,1,1
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19,1,1,1,1
4,4th of July Promo,N.America,2020-07-04,2020-07-11,0,0,0,1
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01,1,1,1,1
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07,1,0,0,0
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03,1,1,1,0
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10,1,1,1,0
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11,0,1,0,0


## Users

In [11]:
users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


In [12]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     58703 non-null  object
 1   first_date  58703 non-null  object
 2   region      58703 non-null  object
 3   device      58703 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [13]:
users.duplicated().sum()

np.int64(0)

In [14]:
users.first_date = pd.to_datetime(users.first_date, errors = 'coerce')
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58703 entries, 0 to 58702
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     58703 non-null  object        
 1   first_date  58703 non-null  datetime64[ns]
 2   region      58703 non-null  object        
 3   device      58703 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.8+ MB


In [15]:
# Conteo por dispositivos
users.device.value_counts()

device
Android    26159
PC         14845
iPhone     11902
Mac         5797
Name: count, dtype: int64

In [16]:
# Conteo por región
users.region.value_counts()

region
EU           43396
N.America     9051
CIS           3130
APAC          3126
Name: count, dtype: int64

In [17]:
users.region = users.region.str.lower().str.replace('.','_')
users.region.unique()

array(['eu', 'n_america', 'apac', 'cis'], dtype=object)

In [18]:
users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,eu,PC
1,F1C668619DFE6E65,2020-12-07,n_america,Android
2,2E1BF1D4C37EA01F,2020-12-07,eu,PC
3,50734A22C0C63768,2020-12-07,eu,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,n_america,iPhone


## User events

In [19]:
events.head()

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


In [20]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     423761 non-null  object 
 1   event_dt    423761 non-null  object 
 2   event_name  423761 non-null  object 
 3   details     60314 non-null   float64
dtypes: float64(1), object(3)
memory usage: 12.9+ MB


In [21]:
events.duplicated().sum()

np.int64(0)

In [22]:
# Comprobamos que los valores no nulos de details, pertenecen a eventos de tipo 'purchase'
len(events[events.event_name == 'purchase'])

60314

In [23]:
null_details = events[events.details.isnull()]
null_details.event_name.unique()


array(['product_cart', 'product_page', 'login'], dtype=object)

In [24]:
# Frecuencias por evento
events.event_name.value_counts()

event_name
login           182465
product_page    120862
purchase         60314
product_cart     60120
Name: count, dtype: int64

In [25]:
# Convertimos columna event_dt a tipo datetime
events.event_dt = pd.to_datetime(events.event_dt, errors = 'coerce')
events.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423761 entries, 0 to 423760
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     423761 non-null  object        
 1   event_dt    423761 non-null  datetime64[ns]
 2   event_name  423761 non-null  object        
 3   details     60314 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 12.9+ MB


In [26]:
events.head()

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


## AB data

In [27]:
ab_data.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,4FF2998A348C484F,A,recommender_system_test


In [28]:
ab_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
dtypes: object(3)
memory usage: 340.6+ KB


In [29]:
ab_data.duplicated().sum()

np.int64(0)

In [30]:
ab_data.ab_test.value_counts()

ab_test
interface_eu_test          10850
recommender_system_test     3675
Name: count, dtype: int64

In [31]:
# Seleccionaremos los usuarios que realizaron la prueba recommender_system_test
recomender_system = ab_data[ab_data.ab_test == 'recommender_system_test']
recomender_system.sample(5)

,user_id,group,ab_test
3512,D9ACCB8EECF03535,A,recommender_system_test
142,1FFFDF2881715DDF,B,recommender_system_test
272,4E5C9F61B66394F7,A,recommender_system_test
1388,FCBE4B0CF081F04D,A,recommender_system_test
2268,8E81C0C7B641D8C2,A,recommender_system_test


In [32]:
recomender_system.group.value_counts()

group
A    2747
B     928
Name: count, dtype: int64

In [33]:
#Checamos si un usuario pertenece a los dos grupos
recomender_system.groupby("user_id")["group"].nunique().gt(1).any()
# True si hay al menos un usuario en más de un grupo


np.False_

## Llevar a cabo el análisis exploratorio de datos:    
- Estudia la conversión en las diferentes etapas del embudo.
- ¿El número de eventos por usuario está distribuido equitativamente entre las muestras?
- ¿Hay usuarios que están presentes en ambas muestras?
- ¿Cómo se distribuye el número de eventos entre los días?
- ¿Hay alguna peculiaridad en los datos que hay que tener en cuenta antes de iniciar la prueba A/B?

In [34]:
# Haremos un merge con nuestra tablas para tener todo en un mismo data frame

users_part = users.merge(
    recomender_system[['user_id', 'group']],
    on = 'user_id',
    how = 'inner'
)

# Ahora hacemos merge con events
events_full = events.merge(
    users_part[['user_id', 'first_date', 'group']],
    on = 'user_id', 
    how = 'inner'
)

events_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23909 entries, 0 to 23908
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     23909 non-null  object        
 1   event_dt    23909 non-null  datetime64[ns]
 2   event_name  23909 non-null  object        
 3   details     3210 non-null   float64       
 4   first_date  23909 non-null  datetime64[ns]
 5   group       23909 non-null  object        
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 1.1+ MB


In [35]:
# Filtraremos a los usuarios que tuvieron eventos 14 días despúes de la first_date de eventos
# creamos máscara
mask = (
    (events_full['event_dt'] >= events_full.first_date) & 
    (events_full.event_dt <= events_full.first_date + pd.Timedelta(days=14))
)
#aplicamos filtro
events_window = events_full[mask].copy()
events_window.head()

print("Eventos antes del filtro:", events_full.shape[0])
print("Eventos después del filtro:", events_window.shape[0])

Eventos antes del filtro: 23909
Eventos después del filtro: 23151


In [ ]:
# Queremos verificar que todos los usuarios hayan hecho login
# Contamos cuántos usuarios únicos tenemos
events_window.user_id.nunique()

3675

In [ ]:
# Vemos cuántos usuarios únicos hicieron login
events_window.groupby('event_name')['user_id'].nunique()

event_name
login           3674
product_cart    1079
product_page    2303
purchase        1128
Name: user_id, dtype: int64

In [38]:
# Construiremos un embudo de las etapas

# 1) Pivot table: contar eventos por usuario y tipo
user_event_counts = events_window.groupby(['user_id','event_name']).size().unstack(fill_value=0)

# 2) Crear flags 0/1 para cada etapa del embudo
has_page = (user_event_counts.get('product_page', 0) > 0).astype(int).rename('has_product_page')
has_cart = (user_event_counts.get('product_cart', 0) > 0).astype(int).rename('has_product_cart')
has_purchase = (user_event_counts.get('purchase', 0) > 0).astype(int).rename('has_purchase')

# 3) Unir flags en un solo DataFrame
flags = pd.concat([has_page, has_cart, has_purchase], axis=1).reset_index()

# 4) Unir con info de usuario (grupo y first_date)
users_part = users.merge(recomender_system[['user_id','group']], on='user_id', how='inner')
df_funnel = users_part.merge(flags, on='user_id', how='left').fillna(0)

# Asegurarnos de que los flags sean enteros
for c in ['has_product_page','has_product_cart','has_purchase']:
    df_funnel[c] = df_funnel[c].astype(int)

# 5) Ver primeras filas
print(df_funnel.head())


            user_id first_date region   device group  has_product_page  \
0  D72A72121175D8BE 2020-12-07     eu       PC     A                 1   
1  DD4352CDCF8C3D57 2020-12-07     eu  Android     B                 1   
2  831887FE7F2D6CBA 2020-12-07     eu  Android     A                 0   
3  4CB179C7F847320B 2020-12-07     eu   iPhone     B                 1   
4  29C92313A98B1176 2020-12-07   apac  Android     B                 1   

   has_product_cart  has_purchase  
0                 0             0  
1                 0             0  
2                 1             1  
3                 0             0  
4                 0             0  
